In [1]:
# Dependencies and Setup
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

from config import username, password, host, database

In [2]:
# File path
global_filepath = '../output_file/mmr_global.csv'

# Read in csv file
mmr_global_df = pd.read_csv(global_filepath)

# Display dataframe
mmr_global_df.head()

,name,id,mmr,ranking,category,latitude,longitude
0,Belarus,BY,2,1,Very Low (<100),53.7440,27.9470
1,Italy,IT,2,1,Very Low (<100),45.3889,10.7776
2,Norway,NO,2,1,Very Low (<100),60.5366,7.4496
3,Poland,PL,2,1,Very Low (<100),52.1484,19.3110
4,Czech Republic,CZ,3,5,Very Low (<100),49.8360,15.5465


In [3]:
# Country name known as None > change it to string
namibia_index = mmr_global_df.loc[mmr_global_df['name'] == 'Namibia'].index[0]
mmr_global_df.loc[namibia_index, 'id'] = str("NA")

# Check if the id is correct
mmr_global_df.loc[namibia_index, 'id']

'NA'

In [4]:
# Check if id column is not null
mmr_global_df.count()

name         185
id           185
mmr          185
ranking      185
category     185
latitude     185
longitude    185
dtype: int64

### Connect to PostgresSQL database

In [5]:
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [6]:
# Create table schema
class Global(Base):
    __tablename__ = 'mmr_global'
    __table_args__ = {'extend_existing': True}
    name = Column(String(255), primary_key=True)
    id = Column(String(255))
    mmr = Column(Integer)
    ranking = Column(Integer)
    category = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)

In [7]:
# Create Engine
connect_string = f'postgresql://{username}:{password}@{host}/{database}'
engine = create_engine(connect_string)

In [8]:
# Specify table
global_table = [Base.metadata.tables['mmr_global']]

# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine, tables=global_table)

In [9]:
# Store data to SQL database
mmr_global_df.to_sql(name='mmr_global', con=engine, if_exists='append', index=False)

In [10]:
# Check of the table exists in database
mmr_global_sql = pd.read_sql_query('select * from mmr_global', con=engine)
mmr_global_sql.head()

,name,id,mmr,ranking,category,latitude,longitude
0,Belarus,BY,2,1,Very Low (<100),53.7440,27.9470
1,Italy,IT,2,1,Very Low (<100),45.3889,10.7776
2,Norway,NO,2,1,Very Low (<100),60.5366,7.4496
3,Poland,PL,2,1,Very Low (<100),52.1484,19.3110
4,Czech Republic,CZ,3,5,Very Low (<100),49.8360,15.5465


In [11]:
# Check if data consistency
mmr_global_sql.count()

name         185
id           185
mmr          185
ranking      185
category     185
latitude     185
longitude    185
dtype: int64